Reference:

- https://neos-guide.org/content/quadratic-assignment-problem

In [1]:
using JuMP, Gurobi

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1278


In [2]:
# flow matrix
F = [0 3 0 2;
     0 0 0 1;
     0 0 0 4;
     0 0 0 0]

4×4 Array{Int64,2}:
 0  3  0  2
 0  0  0  1
 0  0  0  4
 0  0  0  0

In [3]:
# distances matrix
D = [0  0 53   0;
     22 0 40   0;
     0  0  0  55;
     0  0  0   0]

4×4 Array{Int64,2}:
  0  0  53   0
 22  0  40   0
  0  0   0  55
  0  0   0   0

## Formulacion con constraints

In [4]:
#model = Model(Ipopt.Optimizer)
#model = Model(COSMO.Optimizer) # requires convex objective function
model = Model(Gurobi.Optimizer)

n = size(F, 1)
#@variable(model, 0 <= x[1:n, 1:n] <= 1) # real
#@variable(model, 0 <= x[1:n, 1:n] <= 1, Bin) # force binary
@variable(model, x[1:n, 1:n], Bin) # force binary

Academic license - for non-commercial use only - expires 2021-01-11


4×4 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]

Interpretamos $x_{ij}$ como $i$ siendo el indice de la facility y $j$ siendo el indice de la location. $i, j$ varian entre $1$ y $4$.

In [5]:
@constraint(model, sum(x, dims=1) .== 1)
@constraint(model, sum(x, dims=2) .== 1);

In [6]:
function costo(x, F, D) 
    n = size(x, 1)
    s = 0
    for i = 1:n
        for j = 1:n
            i == j && continue
            for h = 1:n
                for k = 1:n
                    k == h && continue
                    s += F[i, j] * D[h, k] * x[i, h] * x[j, k]
                end
            end
        end
    end
    s = s / 2
end

costo (generic function with 1 method)

In [12]:
s = costo(x, F, D)
@objective(model, Min, s)

79.5 x[1,1]*x[2,3] + 33 x[1,2]*x[2,1] + 60 x[1,2]*x[2,3] + 82.5 x[1,3]*x[2,4] + 53 x[1,1]*x[4,3] + 22 x[1,2]*x[4,1] + 40 x[1,2]*x[4,3] + 55 x[1,3]*x[4,4] + 26.5 x[2,1]*x[4,3] + 11 x[2,2]*x[4,1] + 20 x[2,2]*x[4,3] + 27.5 x[2,3]*x[4,4] + 106 x[3,1]*x[4,3] + 44 x[3,2]*x[4,1] + 80 x[3,2]*x[4,3] + 110 x[3,3]*x[4,4]

In [13]:
optimize!(model)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x586e0da4
Model has 16 quadratic objective terms
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 87.5

Presolve time: 0.00s
Presolved: 24 rows, 32 columns, 80 nonzeros
Variable types: 0 continuous, 32 integer (32 binary)

Root relaxation: objective 0.000000e+00, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 1 nodes (6 

In [14]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [15]:
objective_value(model)

0.0

In [16]:
# optimal values
M = value.(x)

4×4 Array{Float64,2}:
  1.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0   1.0
  0.0  -0.0   1.0  -0.0
 -0.0   1.0  -0.0  -0.0

## Formulacion general del quadratic assignment

In [32]:
function quadratic_assignment(; flows, distances)
    model = Model(Gurobi.Optimizer)
    n = size(flows, 1)
    @variable(model, x[1:n, 1:n], Bin)
    @constraint(model, sum(x, dims=1) .== 1)
    @constraint(model, sum(x, dims=2) .== 1)
    s = costo(x, F, D)
    @objective(model, Min, s)
    optimize!(model)
    value.(x), solve_time(model)
end

quadratic_assignment (generic function with 1 method)

In [34]:
M, t = quadratic_assignment(flows=F, distances=D)

Academic license - for non-commercial use only - expires 2021-01-11
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x586e0da4
Model has 16 quadratic objective terms
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 87.5000000
Presolve time: 0.00s
Presolved: 24 rows, 32 columns, 80 nonzeros
Variable types: 0 continuous, 32 integer (32 binary)

Root relaxation: objective 0.000000e+00, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.000

([1.0 -0.0 -0.0 0.0; -0.0 -0.0 -0.0 1.0; 0.0 -0.0 1.0 -0.0; -0.0 1.0 -0.0 -0.0], 0.002627134323120117)

In [36]:
M

4×4 Array{Float64,2}:
  1.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0   1.0
  0.0  -0.0   1.0  -0.0
 -0.0   1.0  -0.0  -0.0

In [38]:
t # tiempo de resolucion en segundos

0.002627134323120117

## Formulacion como un problema QUBO

TO-DO